In [84]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import requests
from bs4 import BeautifulSoup

In [85]:
import pandas as pd
key= pd.read_csv("key.txt")

In [86]:
llm= OpenAI(temperature =0.9 , max_tokens=500)

In [87]:
urls=["https://www.livemint.com/news/india/uttarkashi-tunnel-collapse-silkyara-ndrf-deploys-team-rescue-mission-41-workers-top-10-updates-pushkar-singh-dhami-11700703183514.html",
     "https://www.indiatvnews.com/sports/cricket/india-s-schedule-after-world-cup-2023-final-loss-india-vs-australia-t20i-series-t20-world-cup-2024-2023-11-20-903594"]
responses=[]

for url in urls:
    response=requests.get(url)
    if response.status_code==200:
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        responses.append((text, response.url))
        
    else:
        print(f"Failed to retrieve the URL. Url={url} Status_code :{response.status_code}")

In [88]:
len(responses)

2

In [89]:
import string 
punct=string.punctuation
seps=[]
for i in punct:
    seps.append(i)

In [90]:
text_splitter=RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "?","." ]+seps+[" "],
    chunk_size=1000,
    chunk_overlap=200
)

In [97]:
splitted_tuple=[]
for i in responses:
    splitted_text= text_splitter.split_text(i[0])
    for j in splitted_text:
        splitted_tuple.append((j, i[1]))

In [72]:
import numpy as np
data = np.array([np.array(t) for t in splitted_tuple], dtype=np.float32)

# Initialize the IndexFlatL2 index
index = faiss.IndexFlatL2(data.shape[1])

ValueError: could not convert string to float: 'Uttarkashi Tunnel Collapse: NDRF team enters tunnel to save 41 trapped workers with ambulances on standby | 10 updates | Mint'

In [81]:
documents = [{'page_content': text, 'url': url} for text, url in splitted_tuple]

In [82]:
documents[0]

{'page_content': 'Uttarkashi Tunnel Collapse: NDRF team enters tunnel to save 41 trapped workers with ambulances on standby | 10 updates | Mint',
 'url': 'https://www.livemint.com/news/india/uttarkashi-tunnel-collapse-silkyara-ndrf-deploys-team-rescue-mission-41-workers-top-10-updates-pushkar-singh-dhami-11700703183514.html'}

In [83]:
embeddings=OpenAIEmbeddings()

vector_index_openai=FAISS.from_documents(documents, embeddings)

AttributeError: 'dict' object has no attribute 'page_content'